In [1]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer,
    default_data_collator
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


/home/wanted-1/miniconda3/envs/whisper/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device: cuda:0


In [2]:
import os
import re
import glob
import zipfile
import json
import pandas as pd
from datasets import load_dataset, Features, Value

def preprocess_json(json_path):
    """
    JSON 파일을 로드하여 전처리합니다.
    
    - 상위에 "documents" 키가 있을 경우, 각 문서에서 "text" 필드를 
      평탄화하여 하나의 문자열로 합치고, "abstractive" 필드를 요약으로 사용합니다.
    - 최종적으로 "documents"와 "summary" 두 개의 컬럼을 갖는 DataFrame을 생성합니다.
    - 결측값을 제거한 후, 전처리된 데이터를 JSON 파일에 다시 저장합니다.
    """
    try:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)
        
        # JSON 구조가 딕셔너리이고 "documents" 키가 있는 경우 처리
        if isinstance(data, dict) and "documents" in data:
            docs = data["documents"]  # 각 문서가 담긴 리스트
            new_data = []
            for doc in docs:
                # 1. 문서 텍스트 추출: "text" 필드는 여러 그룹(리스트)로 구성되어 있음
                doc_sentences = []
                if "text" in doc and isinstance(doc["text"], list):
                    for group in doc["text"]:
                        # group은 리스트인데, 각 원소는 문장(dict)입니다.
                        for sentence_item in group:
                            sentence = sentence_item.get("sentence", "")
                            doc_sentences.append(sentence)
                    doc_text = " ".join(doc_sentences).strip()
                else:
                    doc_text = ""
                
                # 2. 요약 추출: "abstractive" 필드가 있으면 사용 (리스트인 경우 모두 합침)
                if "abstractive" in doc and doc["abstractive"]:
                    if isinstance(doc["abstractive"], list):
                        summary_text = " ".join(doc["abstractive"]).strip()
                    else:
                        summary_text = str(doc["abstractive"])
                else:
                    summary_text = ""
                
                new_data.append({
                    "documents": doc_text,
                    "summary": summary_text
                })
            df = pd.DataFrame(new_data)
        # 만약 상위에 "data" 키가 있다면
        elif isinstance(data, dict) and "data" in data:
            df = pd.DataFrame(data["data"])
        elif isinstance(data, list):
            df = pd.DataFrame(data)
        else:
            df = pd.DataFrame(data)
        
        # 확인: "documents"와 "summary" 컬럼이 반드시 존재해야 함
        if "documents" not in df.columns:
            raise ValueError(f"'documents' 필드가 존재하지 않습니다: {json_path}")
        if "summary" not in df.columns:
            raise ValueError(f"'summary' 필드가 존재하지 않습니다: {json_path}")
        
        # 두 컬럼을 문자열로 변환
        df["documents"] = df["documents"].astype(str)
        df["summary"] = df["summary"].astype(str)
        
        # 결측값 제거
        df.dropna(inplace=True)
        
        # 전처리된 데이터를 다시 JSON 파일로 저장 (records 형식)
        df.to_json(json_path, orient="records", force_ascii=False)
    
    except Exception as e:
        raise ValueError(f"⚠️ JSON 파일 {json_path} 전처리 중 오류 발생: {e}")

def load_and_build_dataset_from_zip(zip_path, extracted_dir):
    """
    zip 파일 내의 JSON 파일들을 동적으로 검색하여 데이터셋을 구성합니다.
    
    1. 지정된 경로에 압축 파일을 해제합니다.
    2. glob 모듈을 사용해 모든 JSON 파일을 검색합니다.
    3. 정규표현식을 활용하여 파일명에 'train' (대소문자 구분 없이)이 포함된 파일만 필터링합니다.
    4. 필터링된 파일들에 대해 전처리를 수행한 후, Hugging Face의 load_dataset 함수로 데이터셋을 구성합니다.
    
    :param zip_path: 압축 파일의 경로
    :param extracted_dir: 압축 해제할 디렉토리
    :return: 구성된 데이터셋
    """
    # 압축 해제할 디렉토리가 없으면 생성합니다.
    if not os.path.exists(extracted_dir):
        os.makedirs(extracted_dir)
    
    # zip 파일을 열어 지정된 디렉토리에 압축 해제
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extracted_dir)
    
    # extracted_dir 내의 모든 JSON 파일 검색
    json_files = glob.glob(os.path.join(extracted_dir, "*.json"))
    
    # 파일명에 'train'이 포함된 파일만 필터링 (대소문자 무시)
    train_files = [f for f in json_files if re.search(r"train", os.path.basename(f), re.IGNORECASE)]
    
    if not train_files:
        raise ValueError("압축 해제된 디렉토리에서 'train'이 포함된 JSON 파일을 찾을 수 없습니다.")
    
    # validation 파일도 필요하면 비슷한 방식으로 필터링 가능 (예시)
    validation_files = [f for f in json_files if re.search(r"validation", os.path.basename(f), re.IGNORECASE)]
    
    # 각 train/validation 파일에 대해 전처리 적용
    for json_path in train_files:
        preprocess_json(json_path)
    if validation_files:
        for json_path in validation_files:
            preprocess_json(json_path)
    
    # 데이터 파일 딕셔너리 생성: load_dataset은 리스트로 전달받은 여러 파일들을 자동으로 병합합니다.
    data_files = {"train": train_files}
    if validation_files:
        data_files["validation"] = validation_files
    
    # features를 명시하여 각 필드를 문자열로 강제 지정 (데이터 타입 문제 해결)
    features = Features({
        "documents": Value("string"),
        "summary": Value("string"),
    })
    
    # JSON 파일들로부터 데이터셋 구성
    dataset = load_dataset("json", data_files=data_files, features=features)
    return dataset


In [3]:

# 사용 예시:
zip_path = "/home/wanted-1/potenup-workspace/Project/project3/team2/학습데이터/06_문서요약 텍스트/Training/신문기사_train_original.zip"
extracted_dir = "/home/wanted-1/potenup-workspace/Project/project3/team2/학습데이터/06_문서요약 텍스트/Training/extracted"
dataset = load_and_build_dataset_from_zip(zip_path, extracted_dir)
print(dataset)


Generating train split: 243983 examples [00:07, 34214.49 examples/s]

DatasetDict({
    train: Dataset({
        features: ['documents', 'summary'],
        num_rows: 243983
    })
})


In [5]:
# 전처리된 데이터에서 원문과 요약을 직접 가져옵니다.
all_originals = dataset['train']['documents']
all_summaries = dataset['train']['summary']

# for idx, (orig, summ) in enumerate(zip(all_originals, all_summaries))
    # print(f"Document {idx+1}")
    # print("Original Text:")
    # print(orig)
    # print("Summary:")
    # print(summ)
    # print("-" * 40)

In [6]:
import pandas as pd

# 예를 들어, 전체 문서에서 추출한 원문과 요약문 리스트가 있다고 가정합니다.
data = {
    "original_text": all_originals,
    "summary_text": all_summaries
}
df = pd.DataFrame(data)

# DataFrame 내용 확인
# print(df.head())

# CSV 파일로 저장하기
df.to_csv("extracted_documents_신문기사.csv", index=False, encoding="utf-8-sig")
